In [1]:
import sklearn
import numpy as np
from spacy.lang.fr import French
parser = French()
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import textacy
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import spacy

In [2]:
nlp=spacy.load("fr_core_news_md")

In [33]:
X1=pd.read_csv("data.csv")



In [34]:
X1.shape

(642, 4)

In [16]:
X1.head(10)

,article,date,lien,title
0,"Abid Briki, président de Tunisie en Avant a ét...",04 Juillet 2019 17:36,https://www.mosaiquefm.net/fr/actualite-politi...,"Abid Briki, candidat de ''Tunisie en Avant'' à..."
2,"Mohamed Ennaceur, président de l'Assemblée des...",05 Juillet 2019 15:27,https://www.mosaiquefm.net/fr/actualite-politi...,Election des membres de la Cour Constitutionne...
3,Le président du conseil de la Choura d’Ennahdh...,06 Juillet 2019 23:36,https://www.mosaiquefm.net/fr/actualite-politi...,Harouni : Chahed restera ou partira selon nos ...
4,"La nouvelle formation politique ""Au cœur de la...",08 Juillet 2019 21:26,https://www.mosaiquefm.net/fr/actualite-politi...,Le parti de Nabil Karoui dénonce la décision d...
5,Lotfi Zitoun a posté sur sa page sur Facebook ...,08 Juillet 2019 13:27,https://www.mosaiquefm.net/fr/actualite-politi...,Zitoun abandonne son poste de conseiller polit...
6,Le mouvement Tahya Tounes a estimé que le pays...,05 Juillet 2019 21:06,https://www.mosaiquefm.net/fr/actualite-politi...,Tahya Tounes : Le pays a surmonté la phase d'h...
7,Le président du conseil de la choura d’Ennahdh...,07 Juillet 2019 00:19,https://www.mosaiquefm.net/fr/actualite-politi...,Harouni : Ghannouchi est notre candidat nature...
8,article,date,lien,title
9,"\r\r\n\tLe secrétaire général de l’UGTT, Noure...",19-06-2019,https://www.tuniscope.com/article/178833/actua...,certaines instances indépendantes deviennent d...
10,\r\r\n\tL’Assemblée des représentants du peupl...,19-06-2019,https://www.tuniscope.com/article/178867/actua...,12 députés signent une pétition contre l'amend...


In [4]:
spacy_stopwords = spacy.lang.fr.stop_words.STOP_WORDS
stop=list(spacy_stopwords)

In [5]:
def spacy_tokenizer(sentence):
    sentence=textacy.preprocess_text(sentence,lowercase=True,no_punct=True,no_urls=True
                                    ,no_emails=True,no_currency_symbols=True)
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word.strip().lower() for word in mytokens if word.strip().lower() not in stop ]
    return mytokens

In [6]:
X1=X1.dropna(axis=0)

In [7]:
X1.columns

Index(['article', 'date', 'lien', 'title'], dtype='object')

In [8]:
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [21]:
from sklearn.cluster import KMeans
classifier = kmeans = KMeans(n_clusters=3, random_state=0)
pipe=Pipeline([("vectorizer",tfvectorizer),("classifier",classifier)])

In [30]:
pipe.fit(X1["title"])

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=...3, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0))])

In [24]:

test2="nahdha"
test3="youssef chahed"
test4="mahdi jooma"
pred1=pipe.predict([test4,test2,test3])
pred1

array([2, 2, 2], dtype=int32)

In [31]:
for col,title in zip(X1["article"],X1["title"]):
    predx=pipe.predict([col])
    print(title,predx)

Abid Briki, candidat de ''Tunisie en Avant'' à la présidentielle [0]
Election des membres de la Cour Constitutionnelle:Présence obligatoire [1]
Harouni : Chahed restera ou partira selon nos conditions [0]
Le parti de Nabil Karoui dénonce la décision du pôle judiciaire [1]
Zitoun abandonne son poste de conseiller politique de Ghannouchi  (Photos) [0]
Tahya Tounes : Le pays a surmonté la phase d'hésitation [1]
Harouni : Ghannouchi est notre candidat naturel à la présidentielle [0]
title [0]
certaines instances indépendantes deviennent douteuses, dcélare Taboubi [1]
12 députés signent une pétition contre l'amendement de la loi électorale [1]
Photo du jour : Béji Caid Essebsi reprend ses fonctions [0]
La réforme de la constitution pour avantager certains acteurs politiques selon Jawhar Ben Mbarek [1]
Ce que l'on sait sur la rencontre entre Béji Caïd Essebsi et Nabil Baffoun [1]
Le mouvement Echaâb annonce l'intention d'intenter un recours contre l'amendement de la loi électorale [1]
Lamia 

Intox, rumeurs et accusations réciproques : Des députés appellent à enquêter sur le fameux «jeudi noir» [1]
Encore un conseil municipal en crise : La maire de Nabeul jette l'éponge [0]
Officiel: L'ambassade américaine à Tunis reporte la réception du 4-Juillet à septembre prochain [1]
Etat du président Caïd Essebsi : Me Abdellah trompe les Tunisiens en diffusant une photo «truquée» [1]
Après avoir quitté Nidaa Tounes, Salma Elloumi fait de beaux rêves... [0]
Auréolé du Grand Prix de la biographie, Bertrand Le Gendre présente son ‘‘Bourguiba’’ à Tunis [1]
Salem Labiadh: BCE a sauvé la Tunisie [1]
Neji Jalloul candidat à la présidentielle [1]
Présumée intoxication du président : Saïda Garrache recadre (sèchement) Slim Riahi [1]
Faycel Tebbini affirme connaître les députés derrière la rumeur du décès du président [1]
Après avoir annoncé sa démission de Nidaa Tounes, Lamia Mlih rejoint Tahya Tounes [0]
Plusieurs personnalités rejoignent le parti de Nabil Karoui [0]
Lancement de l’initiative

Afek Tounes contre l’amendement de la loi électorale  [1]
Machroûu-Nidaa : Les concertations à un stade avancé [0]
Législatives: Charfeddine dans une liste indépendante? [0]
Officiel: Al Moubadra et Tahya Tounes fusionnent [0]
Selma Elloumi élue présidente de Nidaa Tounes (Clan Toubel) [1]
Abir Moussi - Ben Ali : Qu'en pense Rached Ghannouchi ? [0]
Prorogation de l'enregistrement des électeurs: Quelle incidence ? [1]
Le clan de Sofiène Toubel réagit à la vidéo polémique [0]
Le Président de la République adresse une lettre à la « famille diplomatique tunisienne » suite à l’élection de la Tunisie au Conseil de sécurité [1]
Al Badil Ettounsi lance une vaste consultation nationale pour sélectionner ses candidats aux élections législatives [1]
Chahed s’entretient à Genève avec Dimitri Medvedev, Président du Conseil des ministres de la fédération de Russie et son homologue marocain Saadeddine Othmani [1]
Youssef Chahed préside un CMR consacré à l’Energie et aux Mines [0]
Après les attentats 

Retour sur les mesures prises par le Conseil ministériel restreint en faveur du secteur des transports [1]
Front populaire : La grande trahison de Hamma Hammami [1]
Municipale partielle au Bardo : Rejet définitif des 2 listes Nidaa Tounes [0]
Hamma Hammami veut empêcher les députés démissionnaires du bloc Front populaire de garder le même nom [0]
Nabil Karoui et son parti (qui n’existe pas) en tête des intentions de votes pour la présidentielle et les législatives [0]
Le célèbre tortionnaire Hassen Abid décède dans l’impunité et l’indignité [1]
Ennahdha aux portes des lycées pour… «encourager» les candidats au bac (photos) [1]
Ben Arous : Le SG du bureau régional de Machrou Tounes dément la démission des 11 dirigeants [1]
Un vendeur ambulant agresse le président de l’arrondissement municipal de l’Ariana, Mohamed Hamaied [1]
Présidentielle : Kaïs Saied affirme qu'il ne votera même pas pour lui même ! [1]
Pour avoir sali la mémoire de Chokri Belaïd, le député Imed Daïmi poursuivi en just

Hamma Hammami: » Les libertés individuelles sont toujours menacées en Tunisie » [1]
Ahmed Seddik : « le bloc parlementaire du Front Populaire va exploser » [0]
Harouni: « Chahed doit s’expliquer quant à sa candidature à la présidentielle » [0]
Jhinaoui: « Pas de normalisation avec Israël » [1]
SOS villages d’enfants: Chahed remercie les Tunisiens pour leur solidarité [1]
Hausses des cotisations sociales : le ministère des Affaires Sociales précise [1]
Le FP riposte aux propos calomnieux de Daïmi à l’encontre de Belaïd [0]
Interdiction aux hommes de médias de faire de la politique: Marzouki s’indigne [1]
Ennahdha a peur de l’Europe et des Etats-Unis selon Jourchi [0]
Rencontre entre BCE et Slim Riahi en Arabie Saoudite ? [1]
Imed Daïmi s’excuse après ses propos polémiques sur le martyr Chokri Belaïd [0]
Amendement de la loi électorale: l’ARP a tranché [1]
Lâarayedh: « Ennahdha mise beaucoup sur les législatives » [0]
Vers un recours contre l’inconstitutionnalité des révisions de la loi 

Accusé d’avoir voulu prendre le pouvoir, Abdelfattah Mourou réagit [1]
Un nouveau parti politique dans le brouhaha des partis [0]
Le dossier médical de BCE bientôt dévoilé au public [1]
Ghannouchi adresse un message à ceux qui se sont félicités de la défaite d’Erdogan [0]
Mourou: la présidence de la République ne m’intéresse pas [1]
Cour constitutionnelle: Ennahdha propose une « solution » [1]
Présidentielle: H. Hammami dit être le candidat du FP [0]
A Kasserine, le Parti Destourien Libre secoué par une démission collective [0]
Vidéo : pour sa première apparition publique après son malaise, BCE se veut déterminé [1]
Imed Daïmi veut un logement social à Paris selon Mohamed Boughaleb [0]
Ghannouchi, candidat « naturel » d’Ennahdha à la présidentielle [0]
Le médecin de BCE démissionne du Courant Démocratique [1]
635 MDT pour réhabiliter les quartiers populaires : pourquoi avoir attendu si longtemps ? [1]
Salem Labiadh: BCE a sauvé la Tunisie [1]
Plusieurs personnalités rejoignent le parti

In [29]:
wc=[]
for i in range(1,20):
    km=KMeans(n_clusters=i,max_iter=300,n_init=10,random_state=0,verbose=True)
    pipe1=Pipeline([("vectorizer",tfvectorizer),("classifier",km)])
    pipe1.fit(X1["article"])
    wc.append(kmeans.inertia_) 
wc

Initialization complete
Iteration  0, inertia 1158.895
Iteration  1, inertia 602.016
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.171393e-08
Initialization complete
Iteration  0, inertia 1223.390
Iteration  1, inertia 602.016
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.171393e-08
Initialization complete
Iteration  0, inertia 1174.392
Iteration  1, inertia 602.016
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.171393e-08
Initialization complete
Iteration  0, inertia 1220.297
Iteration  1, inertia 602.016
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.171393e-08
Initialization complete
Iteration  0, inertia 1221.263
Iteration  1, inertia 602.016
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.171393e-08
Initialization complete
Iteration  0, inertia 1199.884
Iteration  1, inertia 602.016
Converged at iteration 1: center shift 0.000000e+00 within tolerance 1.171393e-0

Iteration  5, inertia 589.905
Iteration  6, inertia 589.696
Iteration  7, inertia 589.545
Iteration  8, inertia 589.498
Iteration  9, inertia 589.447
Iteration 10, inertia 589.438
Converged at iteration 10: center shift 0.000000e+00 within tolerance 1.171393e-08
Initialization complete
Iteration  0, inertia 1138.784
Iteration  1, inertia 593.480
Iteration  2, inertia 592.191
Iteration  3, inertia 591.459
Iteration  4, inertia 590.938
Iteration  5, inertia 590.633
Iteration  6, inertia 590.294
Iteration  7, inertia 590.176
Iteration  8, inertia 590.158
Iteration  9, inertia 590.117
Iteration 10, inertia 590.038
Iteration 11, inertia 589.994
Iteration 12, inertia 589.987
Converged at iteration 12: center shift 0.000000e+00 within tolerance 1.171393e-08
Initialization complete
Iteration  0, inertia 1125.248
Iteration  1, inertia 593.456
Iteration  2, inertia 591.612
Iteration  3, inertia 591.064
Iteration  4, inertia 590.839
Iteration  5, inertia 590.532
Iteration  6, inertia 590.378
Iter

Iteration  4, inertia 577.513
Iteration  5, inertia 576.992
Iteration  6, inertia 576.687
Iteration  7, inertia 576.562
Iteration  8, inertia 576.528
Iteration  9, inertia 576.517
Iteration 10, inertia 576.507
Iteration 11, inertia 576.498
Converged at iteration 11: center shift 0.000000e+00 within tolerance 1.171393e-08
Initialization complete
Iteration  0, inertia 1090.306
Iteration  1, inertia 577.797
Iteration  2, inertia 573.973


KeyboardInterrupt: 

In [32]:
X1.shape

(637, 4)